<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-26 12:16:43
-------------------
qualified stocks: 93
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.39 L
Current:  1.36 C
-------------------
Today PnL: -90.03 K (-0.66%)
Current PnL: -19.01 L (-12.93%)
CY Booked + Current PnL: -10.28 L (-6.99%)
-------------------
Total profit:  3.40 L
Total loss:  -22.42 L
-------------------
Total Booked + Current PnL: 16.70 L (13.91%)
Total Booked PnL: 35.71 L (29.75%)
Curr Year Booked PnL: 8.73 L (6.4%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 78.13 L (57.32%)
Deployed:  1.20 C
Current:  1.36 C
CAGR/XIRR %: 8.03%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-18.25,H-LC,7.37,113056.0,8744.0,7903.0,-1.29,8.38,6.99,15.96,9.0,1.11,0.85,26.70,X5K,ATH,METALS
37,HINDUNILVR,2413.81,2723.00,-6.25,H-LC,7.74,294739.0,24392.0,10227.0,0.06,9.02,3.47,12.81,24.0,2.39,2.22,23.27,XY25,NTT,FMCG
55,MEDANTA,1087.93,1486.00,12.96,H-MC,14.64,161217.0,35017.0,11156.0,-0.47,27.75,6.92,36.59,108.0,3.14,1.21,43.79,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,7098.95,8285.00,-14.42,H-LC,8.10,170907.0,14730.0,11365.0,-1.05,9.43,6.65,16.71,22.0,1.30,1.29,28.51,X40N,BTT,HEALTHCARE
68,SBIN,760.30,863.00,-13.25,M-LC,4.05,215472.0,13992.0,13230.0,-0.41,6.94,6.14,13.51,60.0,1.06,1.62,20.42,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,KPIGREEN,497.21,731.35,6.33,H-SC,11.11,126466.0,1169.0,57833.0,-1.60,0.93,45.73,47.09,141.0,0.02,0.95,58.28,MH,ATH,POWER
61,RAJOOENG,102.50,143.33,-32.03,H-SC,10.03,102900.0,400.0,40429.0,-1.01,0.39,39.29,39.83,114.0,0.01,0.77,1.88,AR,ATH,MISC
56,NATIONALUM,189.63,247.44,-50.29,H-MC,1.95,99916.0,-588.0,31224.0,-0.99,-0.58,31.25,30.49,79.0,-0.02,0.75,31.63,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-39.61,H-LC,3.09,145545.0,-4387.0,18979.0,0.12,-2.93,13.04,9.74,4.0,-0.23,1.09,3.13,X40,NTT,FMCG
53,LTIM,5564.16,7230.2,-4.05,H-LC,3.14,190188.0,-10122.0,70103.0,-0.19,-5.05,36.86,29.94,16.0,-0.14,1.43,31.39,X200,ATH,IT
57,NESTLEIND,1134.30,1377.0,-10.51,H-LC,3.38,275441.0,10015.0,46770.0,2.09,3.77,16.98,21.40,11.0,0.21,2.07,10.67,XY25,NTT,FMCG
21,CIPLA,1492.70,1795.0,-18.41,H-LC,4.24,217392.0,12892.0,28522.0,-0.90,6.30,13.12,20.25,10.0,0.45,1.64,15.59,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-12.67,H-LC,6.47,170456.0,-30930.0,118552.0,0.55,-15.36,69.55,43.51,27.0,-0.26,1.28,18.65,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SFL,1016.35,1287.00,43.89,M-SC,10.26,190004.0,-72214.0,142047.0,3.67,-27.54,74.76,26.63,229.0,-0.51,1.43,18.13,XY24,NTT,MISC
16,BRITANNIA,5502.66,6362.67,14.35,H-LC,8.69,206748.0,8652.0,22308.0,3.47,4.37,10.79,15.63,26.0,0.39,1.56,27.17,XY25,ATH,FMCG
57,NESTLEIND,1134.30,1377.00,-10.51,H-LC,3.38,275441.0,10015.0,46770.0,2.09,3.77,16.98,21.40,11.0,0.21,2.07,10.67,XY25,NTT,FMCG
73,SONACOMS,578.05,1006.00,-35.37,M-SC,14.06,80456.0,-20703.0,95590.0,1.91,-20.47,118.81,74.03,202.0,-0.22,0.61,13.45,AR,BTT,AUTO
30,GREENPANEL,375.16,537.00,195.22,M-SC,6.00,131809.0,-51269.0,130254.0,1.87,-28.00,98.82,43.14,230.0,-0.39,0.99,23.98,XY24,NTT,MISC


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SIEMENS,4406.90,7969.85,-4.75,H-LC,33.19,76578.0,-33594.0,122670.0,-3.21,-30.49,160.19,80.85,15.0,-0.27,0.58,12.79,AR,ATH,ELECTRICAL
74,STARHEALTH,551.56,761.00,4.85,H-SC,2.15,155890.0,-44326.0,120347.0,-3.17,-22.14,77.20,37.97,144.0,-0.37,1.17,25.17,XY24,NTT,INSURANCE
11,BANDHANBNK,214.25,400.00,-11.26,H-SC,8.49,192931.0,-55599.0,271068.0,-2.99,-22.37,140.50,86.70,151.0,-0.21,1.45,28.80,XY24,NTT,BANKS
88,VALIANTORG,512.64,838.00,-308.36,H-SC,13.12,88465.0,-44821.0,129415.0,-2.73,-33.63,146.29,63.47,139.0,-0.35,0.67,48.29,XR,NTT,CHEMICALS
8,AWL,325.39,485.00,-66.09,H-MC,6.66,208684.0,-59763.0,191447.0,-2.58,-22.26,91.74,49.05,116.0,-0.31,1.57,7.30,XY24,NTT,FMCG


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,96.16,M-SC,8.62,84502.0,-16275.0,16368.0,-1.69,-16.15,19.37,0.09,245.0,-0.99,0.64,11.02,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,85.53,H-SC,15.47,116856.0,-25623.0,45141.0,-1.07,-17.98,38.63,13.70,163.0,-0.57,0.88,37.92,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-19.81,H-SC,3.04,114561.0,-29242.0,55986.0,-1.58,-20.33,48.87,18.60,149.0,-0.52,0.86,26.42,OX40N,NTT,CERAMICS
43,INDIGOPNTS,1407.73,1408.0,109.82,M-SC,1.70,137553.0,-37006.0,37043.0,-1.52,-21.20,26.93,0.02,221.0,-1.00,1.03,19.65,OX40N,NTT,PAINTS
50,KANSAINER,299.63,340.0,-69.25,H-SC,3.64,211473.0,-58194.0,94528.0,0.18,-21.58,44.70,13.47,138.0,-0.62,1.59,6.96,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-25.80,H-LC,15.92,245450.0,-50484.0,144398.0,0.20,-17.06,58.83,31.73,1.0,-0.35,1.85,4.79,X40,BTT,IT
45,INFY,1461.46,2275.00,-16.73,H-LC,11.91,272689.0,11088.0,134545.0,-0.57,4.24,49.34,55.67,3.0,0.08,2.05,10.60,X40,BTT,IT
47,ITC,411.90,452.00,-39.61,H-LC,3.09,145545.0,-4387.0,18979.0,0.12,-2.93,13.04,9.74,4.0,-0.23,1.09,3.13,X40,NTT,FMCG
89,VBL,492.64,671.64,-8.60,H-LC,8.96,277608.0,12568.0,83727.0,-0.12,4.74,30.16,36.33,5.0,0.15,2.09,18.67,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-42.01,H-LC,12.82,197086.0,-19433.0,112339.0,-0.92,-8.98,57.00,42.91,7.0,-0.17,1.48,3.21,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,12.96,H-MC,14.64,161217.0,35017.0,11156.0,-0.47,27.75,6.92,36.59,108.0,3.14,1.21,43.79,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-7.68,H-LC,11.13,230520.0,46039.0,24320.0,-0.44,24.96,10.55,38.14,38.0,1.89,1.73,38.21,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,12.96,H-MC,14.64,161217.0,35017.0,11156.0,-0.47,27.75,6.92,36.59,108.0,3.14,1.21,43.79,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,9.63,H-MC,4.79,189300.0,24630.0,43331.0,-0.60,14.96,22.89,41.28,89.0,0.57,1.42,24.13,X40N,ATH,AC
51,KPIGREEN,497.21,731.35,6.33,H-SC,11.11,126466.0,1169.0,57833.0,-1.60,0.93,45.73,47.09,141.0,0.02,0.95,58.28,MH,ATH,POWER
90,VOLTAS,1278.28,1918.49,-3.04,H-MC,6.73,203370.0,11628.0,84399.0,-0.12,6.06,41.50,50.08,99.0,0.14,1.53,13.27,XY25,ATH,AC
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,14.06,178490.0,27577.0,29736.0,0.71,18.27,16.66,37.97,93.0,0.93,1.34,48.20,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,14.06,178490.0,27577.0,29736.0,0.71,18.27,16.66,37.97,93.0,0.93,1.34,48.20,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-49.79,H-SC,12.37,139931.0,16595.0,117164.0,0.10,13.46,83.73,108.45,119.0,0.14,1.05,38.82,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-43.06,M-SC,5.57,96142.0,4644.0,74106.0,-2.07,5.07,77.08,86.07,223.0,0.06,0.72,39.08,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-11.07,M-LC,7.75,156463.0,5518.0,103938.0,-0.58,3.66,66.43,72.51,53.0,0.05,1.18,9.90,XR,NTT,IT
51,KPIGREEN,497.21,731.35,6.33,H-SC,11.11,126466.0,1169.0,57833.0,-1.60,0.93,45.73,47.09,141.0,0.02,0.95,58.28,MH,ATH,POWER


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-45.67,L-SC,27.05,81888.0,-31661.0,71709.0,-0.79,-27.88,87.57,35.27,268.0,-0.44,0.62,100.42,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-2.99,H-SC,16.17,90231.0,-10732.0,105263.0,-1.14,-10.63,116.66,93.63,148.0,-0.10,0.68,34.40,SR,ATH,CHEMICALS
54,MASFIN,326.60,398.61,-18.70,H-SC,12.55,92550.0,-5430.0,27034.0,-1.30,-5.54,29.21,22.05,152.0,-0.20,0.70,34.53,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-18.25,H-LC,7.37,113056.0,8744.0,7903.0,-1.29,8.38,6.99,15.96,9.0,1.11,0.85,26.70,X5K,ATH,METALS
84,UJJIVANSFB,52.77,60.00,85.53,H-SC,15.47,116856.0,-25623.0,45141.0,-1.07,-17.98,38.63,13.70,163.0,-0.57,0.88,37.92,OX40N,NTT,BANKS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-45.67,L-SC,27.05,81888.0,-31661.0,71709.0,-0.79,-27.88,87.57,35.27,268.0,-0.44,0.62,100.42,XR,NTT,HOTELS
28,DMART,3764.92,5201.00,-7.68,H-LC,11.13,230520.0,46039.0,24320.0,-0.44,24.96,10.55,38.14,38.0,1.89,1.73,38.21,X40N,NTT,FMCG
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,14.06,178490.0,27577.0,29736.0,0.71,18.27,16.66,37.97,93.0,0.93,1.34,48.20,AR,ATH,AUTO
69,SFL,1016.35,1287.00,43.89,M-SC,10.26,190004.0,-72214.0,142047.0,3.67,-27.54,74.76,26.63,229.0,-0.51,1.43,18.13,XY24,NTT,MISC
37,HINDUNILVR,2413.81,2723.00,-6.25,H-LC,7.74,294739.0,24392.0,10227.0,0.06,9.02,3.47,12.81,24.0,2.39,2.22,23.27,XY25,NTT,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.98
1,25,41.68
2,50,71.31


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.19
LC    36.51
MC    26.31
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.35
X40      14.43
X40N     13.98
XY25     11.94
XR        8.98
AR        8.46
OX40N     7.36
X5K       2.27
MH        1.70
X200      1.43
SR        1.11
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.28
H-MC    23.43
H-SC    22.44
M-SC    13.20
M-LC     7.19
M-MC     2.55
L-SC     1.55
L-LC     1.04
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.72,0.48,30.00
IT,12.10,-16.08,74.05
FINANCE,8.00,-22.75,68.13
BANKS,7.44,-15.36,64.34
MISC,5.94,-16.19,82.80
PAINTS,5.51,-19.06,42.52
ELECTRICAL,4.72,-12.01,60.52
AUTO,4.58,-11.13,59.48
AC,3.68,1.18,32.28


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2867254.0,23
XR,1102082.0,13
AR,1083430.0,10
X40,818979.0,11
X40N,508916.0,11
OX40N,479233.0,10
XY25,459971.0,8
SR,196091.0,2
X5K,137417.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2790572.0,27
M-SC,1491115.0,17
H-MC,1262788.0,18
H-LC,1144938.0,19
M-LC,423751.0,5
M-MC,338208.0,2
L-SC,256480.0,3
L-MC,59207.0,1
L-LC,45474.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      945669.0      6
M-SC       XY24      868915.0      7
H-SC       AR        660697.0      5
           XR        530519.0      6
H-LC       X40       510758.0      6
H-MC       XY24      487944.0      5
M-MC       XY24      338208.0      2
H-LC       AR        235009.0      2
H-MC       X40       223302.0      4
H-SC       OX40N     207269.0      4
H-MC       XY25      197855.0      2
H-SC       SR        196091.0      2
           X40N      192494.0      3
H-LC       X40N      192411.0      5
M-LC       XY24      177069.0      2
L-SC       XR        166502.0      2
M-SC       AR        157988.0      2
           OX40N     131381.0      4
M-LC       X5K       129514.0      1
M-SC       XY25      124107.0      1
H-MC       X40N      124011.0      3
M-SC       XR        123805.0      2
H-MC       XR        118111.0      1
M-LC       XR        103938.0      1
L-SC       OX40N      89978.0      1
M-SC       X40        84919.0      1
H-LC       XY25       79305.0      3
           X200       70103.0      1
L-MC       XR         59207.0      1
H-SC       MH         57833.0      1
H-MC       OX40N      50605.0      1
H-LC       XY24       49449.0      1
L-LC       XY25       45474.0      1
H-MC       MH         31224.0      1
           AR         29736.0      1
M-LC       XY25       13230.0      1
H-LC       X5K         7903.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
